In [ ]:
import mlflow
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_aws import ChatBedrockConverse
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from opensearchpy import RequestsHttpConnection, AWSV4SignerAuth

from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer

import sagemaker
import boto3
import subprocess
from os import environ
import streamlit as st
from typing import List
import pandas as pd

In [ ]:
system_prompt_start = """Je bent een hulpvaardige assistent die Nederlands spreekt.
Je krijgt vragen over de catalogus van episodes van het videoplatform VRT MAX. 
Je probeert mensen te helpen om episodes aan te bevelen die aansluiten bij hun vraag.
Daarvoor krijg je een beschrijving van een aantal episodes.
Aan jou om wat relevant is aan te bevelen.
Begin niet met het geven van jouw mening over de episodes. Begin direct met het aanbevelen van relevante content aan de gebruiker.
Leg ook telkens uit waarom je een episode aanbeveelt.
Geef voor alle episodes die je aanbeveelt ook de URL mee als referentie."""

message_start = """Beantwoordt de vraag op basis van de volgende context:

{context}

Vraag: {question}"""

model = "meta.llama3-70b-instruct-v1:0"
temperature = 0.6
top_p = 0.4
max_tokens = 512

params = {
    "system_prompt_start": system_prompt_start,
    "message_start": message_start,
    "model": model,
    "temperature": temperature,
    "top_p": top_p,
    "max_tokens": max_tokens,
}

In [ ]:
# Fix for issue in huggingface embedding, see https://github.com/langchain-ai/langchain/discussions/17773
class CustomEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, documents: List[str]) -> List[List[float]]:
        return [self.model.encode(d).tolist() for d in documents]

    def embed_query(self, query: str) -> List[float]:
        return self.model.encode([query])[0].tolist()

In [ ]:
with mlflow.start_run():
    mlflow.log_params(params)

    # Retriever
    model_name = "NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers"
    encode_kwargs = {'normalize_embeddings': False}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        encode_kwargs=encode_kwargs
    )

    embeddings = CustomEmbeddings(model_name=model_name)
    # LLM

    # Prompt

    # Build chain
    model_info = mlflow.langchain.log_model(chain, "langchain_model")